# Scatter Plots Correlation
- Spearman Correlation
- Scatter Plot Visualization

> **Note:**  
> Use the DATASET TEMPLATES below to run the codes

In [83]:
# DATA_FILE = "raw_data/PSEN1_Sun.csv"
# OUTCOME_COLUMNS = ["AAO", "Total Activity", "Amount of Ab42", "Amount of Ab40", "Ab42/Ab40 ratio"]
# OUTPUT_PREFIX = "PSEN1_Sun"

In [84]:
# DATA_FILE = "raw_data/PSEN1_Petit.csv"
# OUTCOME_COLUMNS = ["Aβ37 [%]", "Aβ38 [%]", "Aβ40 [%]", "Aβ42 [%]", "Aβ43 [%]"]
# OUTPUT_PREFIX = "PSEN1_Petit"

In [85]:
# DATA_FILE = "raw_data/PSEN1_Pillai.csv"
# OUTCOME_COLUMNS = ["Aβ40 (relative to WT)", "Aβ42 (relative to WT)", "Aβ42/40"]
# OUTPUT_PREFIX = "PSEN1_Pillai"

In [86]:
# DATA_FILE = "raw_data/PSEN2_Pillai.csv"
# OUTCOME_COLUMNS = ["Aβ40 (relative to WT)", "Aβ42 (relative to WT)", "Aβ42/40"]
# OUTPUT_PREFIX = "PSEN2_Pillai"

In [87]:
# DATA_FILE = "raw_data/APP_Pillai.csv"
# OUTCOME_COLUMNS = ["Aβ40 (relative to WT)", "Aβ42 (relative to WT)", "Aβ42/40"]
# OUTPUT_PREFIX = "APP_Pillai"

In [88]:
# --- 1️⃣ Import Libraries ---
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

### Change the cell below to read the dataset you want

In [89]:
# --- 2️⃣ User Inputs (EDIT THESE ONLY) ---
DATA_FILE = "raw_data/APP_Pillai.csv"
OUTCOME_COLUMNS = ["Aβ40 (relative to WT)", "Aβ42 (relative to WT)", "Aβ42/40"]
OUTPUT_PREFIX = "APP_Pillai"

In [90]:
# --- 3️⃣ Load Dataset ---
df = pd.read_csv(DATA_FILE)

# Validate AlphaMissense column
if "AlphaMissense_rankscore" not in df.columns:
    raise ValueError("Dataset does not contain 'AlphaMissense_rankscore'.")

In [91]:
# --- 4️⃣ Numeric cleaning helper ---
def coerce_numeric(s: pd.Series) -> pd.Series:
    """Convert to numeric and clean symbols like commas or percent signs."""
    if s.dtype.kind in "biufc":
        return s.astype(float)
    cleaned = (
        s.astype(str)
        .str.replace(r'[%\s]', '', regex=True)
        .str.replace(',', '', regex=False)
        .replace({'nan': np.nan, 'None': np.nan})
    )
    return pd.to_numeric(cleaned, errors="coerce")

# Clean predictor column
x = coerce_numeric(df["AlphaMissense_rankscore"])

In [92]:
# --- 5️⃣ Generate scatter plots ---
sns.set(style="white", font_scale=1.1)

for outcome in OUTCOME_COLUMNS:

    if outcome not in df.columns:
        print(f"⚠ Skipping {outcome}: not found in dataset.")
        continue

    y = coerce_numeric(df[outcome])

    # Compute correlation
    mask = np.isfinite(x.values) & np.isfinite(y.values)
    if mask.sum() < 3:
        print(f"⚠ Not enough valid data for {outcome}. Skipping.")
        continue

    rho, pval = stats.spearmanr(x[mask], y[mask])

    # --- 6️⃣ Create scatter plot ---
    plt.figure(figsize=(6, 5))
    sns.scatterplot(x=x, y=y, s=60, edgecolor="black")

    # regression/lowess trendline (optional but helpful)
    sns.regplot(x=x, y=y, scatter=False, color="red", lowess=True)

    plt.xlabel("AlphaMissense Rank Score", fontsize=12)
    plt.ylabel(outcome, fontsize=12)
    plt.title(f"{outcome} vs AlphaMissense Rankscore", fontsize=14)

    # Annotation of correlation
    plt.text(
        0.05, 0.95,
        f"Spearman ρ = {rho:.3f}\np-value = {pval:.3g}",
        transform=plt.gca().transAxes,
        verticalalignment="top",
        bbox=dict(boxstyle="round", facecolor="white", alpha=0.8)
    )

    plt.tight_layout()

    # --- 7️⃣ Save figure ---
    safe_outcome = outcome.replace("/", "_").replace(" ", "_")
    fig_name = f"figures/{OUTPUT_PREFIX}_scatter_{safe_outcome}.png"
    plt.savefig(fig_name, dpi=600, bbox_inches="tight", facecolor="white")
    plt.close()

    print(f"📊 Saved scatter plot → {fig_name}")

print("\n✅ All scatter plots completed.")


📊 Saved scatter plot → figures/APP_Pillai_scatter_Aβ40_(relative_to_WT).png
📊 Saved scatter plot → figures/APP_Pillai_scatter_Aβ42_(relative_to_WT).png
📊 Saved scatter plot → figures/APP_Pillai_scatter_Aβ42_40.png

✅ All scatter plots completed.
